<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_WordPiece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Алгоритм WordPiece — субсловная токенизация с максимизацией правдоподобия


## 🔹 Введение

**WordPiece** — это алгоритм субсловной токенизации, разработанный Google и впервые применённый в модели **BERT**. Он позволяет эффективно обрабатывать слова, не входящие в словарь (OOV — out-of-vocabulary), разбивая их на семантически значимые подчасти — *субтокены*. В отличие от BPE, WordPiece использует **вероятностный критерий** для выбора пар к объединению — он максимизирует **логарифмическое правдоподобие** корпуса при обучении языковой модели.



## 🔹 Основные отличия от BPE

| Критерий             | BPE                          | WordPiece                          |
|----------------------|------------------------------|------------------------------------|
| Критерий объединения | Наиболее частая пара         | Пара с максимальным score: `freq(pair) / (freq(A) * freq(B))` |
| Префиксы             | Нет                          | `##` для не-начальных субтокенов   |
| Цель                 | Минимизация длины кодирования| Максимизация правдоподобия корпуса |
| Использование        | GPT, RoBERTa                 | BERT, ALBERT, ELECTRA              |



## 🔹 Математическая основа: Почему `score = freq(pair) / (freq(A) * freq(B))`?

### 🔸 Теоретическое обоснование через максимизацию правдоподобия

WordPiece основан на принципе **максимизации правдопоподобия** (Maximum Likelihood Estimation) для языковой модели. Рассмотрим этот вывод подробно.

**Шаг 1: Моделирование вероятности текста**

Предположим, что наш корпус состоит из последовательностей субтокенов. Вероятность корпуса можно выразить через произведение вероятностей отдельных пар (биграммная модель):

$$
P(\text{корпус}) = \prod_{(X,Y)} P(Y | X)^{freq(X,Y)}
$$

где $freq(X,Y)$ — частота пары (X, Y) в корпусе.

**Шаг 2: Логарифмическое правдоподобие**

Для удобства оптимизации переходим к логарифмическому правдоподобию:

$$
\log P(\text{корпус}) = \sum_{(X,Y)} freq(X,Y) \cdot \log P(Y | X)
$$

**Шаг 3: Эффект объединения пар**

Когда мы объединяем пару (A, B) в новый токен AB, мы изменяем структуру корпуса. При этом:

1. Исчезают все вхождения пары (A, B)
2. Появляются новые вхождения токена AB
3. Меняются вероятности переходов

**Шаг 4: Вычисление изменения правдоподобия**

Изменение логарифмического правдоподобия можно аппроксимировать как:

$$
\Delta \log P \approx freq(A,B) \cdot \log \left( \frac{P(AB)}{P(A) \cdot P(B)} \right) - \left[ freq(A) \cdot H_A + freq(B) \cdot H_B \right]
$$

где $H_A$ и $H_B$ — энтропии распределений следующих токенов после A и B соответственно.

**Шаг 5: Упрощение для практического применения**

На практике вторым слагаемым часто пренебрегают, так как:
- Энтропийные члены сложно вычислять на каждом шаге
- Основной вклад вносит первое слагаемое
- Для частых пар соотношение $\frac{P(AB)}{P(A)P(B)}$ является хорошим индикатором

Таким образом, получаем:

$$
\Delta \log P \propto freq(A,B) \cdot \log \left( \frac{P(AB)}{P(A) \cdot P(B)} \right)
$$

**Шаг 6: Переход к частотам**

Если мы оцениваем вероятности через относительные частоты:

$$
P(X) \approx \frac{freq(X)}{N}, \quad P(AB) \approx \frac{freq(AB)}{N}
$$

где N — общее количество токенов, то:

$$
\frac{P(AB)}{P(A)P(B)} \approx \frac{freq(AB) \cdot N}{freq(A) \cdot freq(B)}
$$

Поскольку N — константа для данного корпуса, максимизация:

$$
score(A,B) = \frac{freq(A,B)}{freq(A) \cdot freq(B)}
$$

эквивалентна максимизации изменения правдоподобия.

### 🔸 Статистическая интерпретация

Формула $ \frac{freq(A,B)}{freq(A)freq(B)} $ имеет глубокий статистический смысл:

1. **Взаимная информация**: Score пропорционален точечной взаимной информации (Pointwise Mutual Information) между токенами A и B:
   
   $$
   PMI(A,B) = \log \frac{P(A,B)}{P(A)P(B)} \approx \log score(A,B)
   $$

2. **Индикатор устойчивых сочетаний**: Высокий score указывает на то, что пара встречается значительно чаще, чем если бы токены были независимы. Это характерно для:
   - Морфемных сочетаний ("##ние", "##тель")
   - Устойчивых лингвистических конструкций
   - Семантически связанных пар

3. **Защита от редких объединений**: В отличие от BPE, который может объединять редкие, но частые пары, WordPiece избегает объединения пар с низкой взаимной информацией, даже если они встречаются часто.

### 🔸 Практическая реализация

На практике алгоритм вычисляет score для всех возможных пар и выбирает пару с максимальным значением:

```python
def calculate_score(pair, frequencies):
    A, B = pair
    freq_A = frequencies[A]
    freq_B = frequencies[B]
    freq_AB = frequencies.get(pair, 0)
    
    if freq_A * freq_B == 0:
        return 0
    return freq_AB / (freq_A * freq_B)
```

Это обеспечивает выбор пар, которые:
- Статистически значимо связаны
- Образуют лингвистически осмысленные единицы
- Максимально увеличивают правдоподобие корпуса



## 🔹 Объяснение специальных токенов

В словаре WordPiece (и BERT) всегда присутствуют следующие служебные токены:

- **[UNK]** — *unknown token*. Заменяет любое слово или символ, отсутствующий в словарь.
- **[CLS]** — *classification token*. Ставится в начало последовательности. В задачах классификации именно вектор этого токена используется как представление всего предложения.
- **[SEP]** — *separator token*. Разделяет два предложения (например, в задаче парной классификации: вопрос-ответ, предложение1-предложение2).
- **[MASK]** — *mask token*. Используется в обучении BERT для маскирования части слов — модель учится предсказывать, что было замаскировано.

Эти токены **не участвуют в процессе построения словаря WordPiece**, но **добавляются в финальный словарь вручную** перед обучением модели.



## 🔹 Пошаговый алгоритм WordPiece с вычислениями

Рассмотрим корпус из двух предложений (для простоты):

> "Ученик учится"  
> "Ученик читает"

Целевой размер словаря: `V = 20` (включая спецсимволы).



### 🔸 Шаг 0: Подготовка корпуса

Разбиваем каждое слово на символы, добавляя `##` ко всем символам, кроме первого.

```
Ученик → ['У', '##ч', '##е', '##н', '##и', '##к']
учится → ['у', '##ч', '##и', '##т', '##с', '##я']
читает → ['ч', '##и', '##т', '##а', '##е', '##т']
```

Теперь корпус — это список последовательностей:

```python
corpus = [
    ['У', '##ч', '##е', '##н', '##и', '##к', 'у', '##ч', '##и', '##т', '##с', '##я'],
    ['У', '##ч', '##е', '##н', '##и', '##к', 'ч', '##и', '##т', '##а', '##е', '##т']
]
```



### 🔸 Шаг 1: Инициализация словаря

Собираем все уникальные символы/субтокены:

```python
vocab = {'[UNK]', '[CLS]', '[SEP]', '[MASK]'}  # специальные токены
vocab |= {'У', 'у', 'ч', '##ч', '##е', '##н', '##и', '##к', '##т', '##с', '##я', '##а'}
```

Текущий размер: 16 токенов. Нужно добавить ещё 4.



### 🔸 Шаг 2: Подсчёт частот пар и вычисление score

Пройдём по всем последовательностям и посчитаем частоты **смежных пар**.

Пример для первой последовательности:

```
['У', '##ч'] → 1
['##ч', '##е'] → 1
['##е', '##н'] → 1
['##н', '##и'] → 1
['##и', '##к'] → 1
['##к', 'у'] → 1
['у', '##ч'] → 1
['##ч', '##и'] → 1
['##и', '##т'] → 1
['##т', '##с'] → 1
['##с', '##я'] → 1
```

Аналогично для второй последовательности:

```
['У', '##ч'] → 1 (итого 2)
['##ч', '##е'] → 1 (итого 2)
['##е', '##н'] → 1 (итого 2)
['##н', '##и'] → 1 (итого 2)
['##и', '##к'] → 1 (итого 2)
['##к', 'ч'] → 1
['ч', '##и'] → 1
['##и', '##т'] → 1 (итого 2)
['##т', '##а'] → 1
['##а', '##е'] → 1
['##е', '##т'] → 1
```

Теперь посчитаем **score = freq(pair) / (freq(first) * freq(second))**

Например, возьмём пару `('##ч', '##е')`:

- freq(pair) = 2
- freq('##ч') = 3 (встречается в: Ученик×2, учится×1)
- freq('##е') = 3 (Ученик×2, читает×1)
- score = 2 / (3 * 3) = 2/9 ≈ 0.222

Пара `('##н', '##и')`:

- freq(pair) = 2
- freq('##н') = 2
- freq('##и') = 4 (Ученик×2, учится×1, читает×1)
- score = 2 / (2 * 4) = 2/8 = 0.25 ← **лучше!**

Пара `('##и', '##к')`:

- freq(pair) = 2
- freq('##и') = 4
- freq('##к') = 2
- score = 2 / (4 * 2) = 2/8 = 0.25 ← **тоже 0.25**

Пара `('##и', '##т')`:

- freq(pair) = 2
- freq('##и') = 4
- freq('##т') = 3 (учится×1, читает×2)
- score = 2 / (4 * 3) = 2/12 ≈ 0.166

→ **Максимальный score = 0.25** у пар `('##н', '##и')` и `('##и', '##к')`.

Выбираем первую по алфавиту — `('##н', '##и')`.



### 🔸 Шаг 3: Объединение и обновление корпуса

Объединяем `##н` + `##и` → `##ни`

Обновляем все последовательности:

Было:
```
['У', '##ч', '##е', '##н', '##и', '##к', ...]
```

Стало:
```
['У', '##ч', '##е', '##ни', '##к', ...]
```

Теперь пересчитываем частоты с учётом нового токена.

Добавляем `##ни` в словарь → размер = 17.


### 🔸 Шаг 4: Следующая итерация

Теперь считаем пары с участием `##ни`.

Например, пара `('##е', '##ни')`:

- freq(pair) = 2
- freq('##е') = 3
- freq('##ни') = 2
- score = 2 / (3 * 2) = 1/3 ≈ 0.333 ← **новый максимум!**

Пара `('##ни', '##к')`:

- freq(pair) = 2
- freq('##ни') = 2
- freq('##к') = 2
- score = 2 / (2 * 2) = 0.5 ← **ещё лучше!**

→ Выбираем `('##ни', '##к')` → объединяем в `##ник`

Обновляем корпус:

```
['У', '##ч', '##е', '##ник', ...]
```

Добавляем `##ник` в словарь → размер = 18.



### 🔸 Шаг 5: Продолжаем

Теперь пара `('##е', '##ник')`:

- freq = 2
- freq('##е') = 3
- freq('##ник') = 2
- score = 2/(3*2) = 0.333

Пара `('##ч', '##е')`:

- freq = 2
- freq('##ч') = 3
- freq('##е') = 3
- score = 2/9 ≈ 0.222

Пара `('У', '##ч')`:

- freq = 2
- freq('У') = 2
- freq('##ч') = 3
- score = 2/(2*3) = 1/3 ≈ 0.333

→ Максимум у `('##е', '##ник')` и `('У', '##ч')` — оба 0.333.

Выбираем `('У', '##ч')` → объединяем в `Уч`

Обновляем:

```
['Уч', '##е', '##ник', ...]
```

Добавляем `Уч` → размер = 19.



### 🔸 Шаг 6: Последняя итерация

Теперь пара `('Уч', '##е')`:

- freq = 2
- freq('Уч') = 2
- freq('##е') = 3
- score = 2/(2*3) = 0.333

Пара `('##е', '##ник')` — тоже 0.333

Выбираем `('Уч', '##е')` → `Уче`

Добавляем `Уче` → размер = 20. ✅ Цель достигнута.

Финальный словарь (упрощённо):

```python
{
    '[UNK]', '[CLS]', '[SEP]', '[MASK]',
    'У', 'у', 'ч', '##ч', '##е', '##н', '##и', '##к', '##т', '##с', '##я', '##а',
    '##ни', '##ник', 'Уч', 'Уче'
}
```



## 🔹 Токенизация новых предложений

Теперь попробуем токенизировать:

> "Ученик учится"

Процесс токенизации — **жадный**: идём слева направо, берём максимально возможный токен из словаря.

1. "Уч" — есть → токен `Уч`
2. остаток: "еник учится"
3. "е" — есть как `##е`, но лучше взять `Уче`? Нет, "Уче" уже не подходит, т.к. строка начинается с "е".
4. "е" → `##е`
5. "ни" → `##ни`
6. "к" → `##к` → но у нас есть `##ник`! → проверяем: "еник" → `##е` + `##ник` → есть `##ник` → объединяем → `##е` + `##ник` → но в словаре нет `##еник`, зато есть `##ник` → можно ли взять `##е` + `##ник`? Да, если они идут подряд.

→ Но в словаре у нас есть `Уче` и `##ник`, но нет `##еник`.

Лучше: после `Уч` → остаток "еник" → пробуем "е" → `##е`, затем "ник" → `##ник` → получаем `['Уч', '##е', '##ник']`

Далее: пробел → пропускаем. "учится" → "у" + "чится"

- "у" → есть
- "ч" → нет как отдельного, но есть `##ч` — но он не начальный! → ошибка?

❗ **Важно**: в WordPiece токенизатор **не может начать слово с `##`**. Поэтому если слово начинается с символа, который в словаре есть только с `##`, — это проблема.

Значит, в нашем словаре **не хватает токена "ч" без `##`** — но он у нас есть! В начальном словаре был `'ч'` (из слова "читает").

→ "учится":  
- "у" → есть  
- "ч" → есть (без `##`)  
- "и" → `##и`? Но у нас нет начального "и", только `##и`. → проблема.

❗ Это показывает: **в нашем словаре не хватает базовых символов без `##` для всех возможных начальных букв**.

→ На практике, WordPiece **всегда включает все начальные символы без `##`**, и только внутренние помечаются.

Значит, в нашем случае, слово "учится" токенизируется как:

- "у" → `'у'`
- "ч" → `'ч'` (есть!)
- "и" → нет начального `'и'`, только `'##и'` → значит, берём `'ч'` + `'##и'`? Но `'ч'` и `'##и'` — это разные токены, и в словаре может быть пара `'ч##и'`? Нет.

→ Тогда разбиваем: `'ч'`, `'##и'`, `'##т'`, `'##с'`, `'##я'`

→ Итого: `['у', 'ч', '##и', '##т', '##с', '##я']`

Но у нас в словаре есть `'##ит'`? Нет. А могло бы быть, если бы мы продолжили обучение.



## 🔹 Итоговая токенизация

- "Ученик" → `['Уч', '##е', '##ник']` или `['У', '##ч', '##е', '##ник']` — зависит от того, какой токен длиннее и есть ли он.

В нашем словаре `'Уче'` есть, но после него остаётся `'ник'` — а `'ник'` как отдельный токен у нас есть только как `'##ник'`, который не может начинать слово.

→ Поэтому токенизатор выберет: `'Уч'` + `'##е'` + `'##ник'`

- "учится" → `'у'` + `'ч'` + `'##и'` + `'##т'` + `'##с'` + `'##я'`

→ Полная последовательность:  
`['Уч', '##е', '##ник', 'у', 'ч', '##и', '##т', '##с', '##я']`

Если бы у нас был токен `'##ится'`, было бы лучше.



## 🔹 Почему WordPiece эффективен?

1. **Обрабатывает OOV**: любое слово можно разбить на субтокены.
2. **Семантически осмысленные единицы**: `##ник`, `##тель`, `##ство` — часто соответствуют морфемам.
3. **Контекстная разметка**: `##` позволяет модели понимать, что токен — часть слова.
4. **Математически обоснован**: максимизация правдоподобия > жадная частота.



## 🔹 Заключение

WordPiece — это не просто «BPE с другим score». Это **вероятностный алгоритм**, основанный на языковом моделировании. Он строит словарь, **оптимальный для предсказания текста**, а не просто для сжатия. Специальные токены `[UNK]`, `[CLS]`, `[SEP]`, `[MASK]` играют ключевую роль в архитектуре BERT и не участвуют в построении словаря, но обязательны для финального использования.

In [ ]:
import json
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.pre_tokenizers import Sequence, Whitespace, Punctuation

# -------------------------------
# 1️⃣ Расширенный корпус
# -------------------------------
data = [
    {"text": "Привет, как дела?", "label": "greeting"},
    {"text": "Сегодня хорошая погода.", "label": "weather"},
    {"text": "Я учу машинное обучение.", "label": "education"},
    {"text": "Сегодня я изучаю NLP.", "label": "education"},
    {"text": "Программирование на Python очень интересно.", "label": "education"},
    {"text": "Погода на улице дождливая и холодная.", "label": "weather"},
    {"text": "Привет! Давно не виделись.", "label": "greeting"},
]

with open("small_corpus.jsonl", "w", encoding="utf-8") as f:
    for item in data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

# -------------------------------
# 2️⃣ Создание и обучение WordPiece токенизатора
# -------------------------------
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

# Нормализация
tokenizer.normalizer = Lowercase()

# Препроцессинг: пробелы + разделение пунктуации
tokenizer.pre_tokenizer = Sequence([
    Whitespace(),
    Punctuation()
])

# Trainer с большим словарем для осмысленных токенов
trainer = trainers.WordPieceTrainer(
    vocab_size=200,
    min_frequency=1,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Загружаем тексты
with open("small_corpus.jsonl", "r", encoding="utf-8") as f:
    lines = [json.loads(line)["text"] for line in f]

# Обучаем токенизатор
tokenizer.train_from_iterator(lines, trainer)
tokenizer.save("wordpiece_tokenizer.json")

# -------------------------------
# 3️⃣ Тестирование
# -------------------------------
tokenizer = Tokenizer.from_file("wordpiece_tokenizer.json")

test_texts = [
    "Сегодня я изучаю NLP.",
    "Программирование на Python очень интересно.",
    "Привет, как дела?"
]

for text in test_texts:
    output = tokenizer.encode(text)
    print("\nТекст:", text)
    print("Токены:", output.tokens)
    print("Индексы:", output.ids)